# Getting started

### CLEF 2025 - CheckThat! Lab  - Task 4 Scientific Web Discourse - Subtask 4b (Scientific Claim Source Retrieval)

This notebook enables participants of subtask 4b to quickly get started. It includes the following:
- Code to upload data, including:
    - code to upload the collection set (CORD-19 academic papers' metadata)
    - code to upload the query set (tweets with implicit references to CORD-19 papers)
- Code to run a baseline retrieval model (BM25)
- Code to evaluate the baseline model

Participants are free to use this notebook and add their own models for the competition.

# 1) Importing data

In [16]:
import numpy as np
import pandas as pd
import tqdm
from tqdm.auto import tqdm
import openai
import numpy as np
from sklearn.preprocessing import normalize
from tqdm.auto import tqdm

## 1.a) Import the collection set
The collection set contains metadata of CORD-19 academic papers.

The preprocessed and filtered CORD-19 dataset is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.


In [17]:
# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_COLLECTION_DATA = '../subtask4b_collection_data.pkl' #MODIFY PATH

In [18]:
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [19]:
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [20]:
df_collection.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


## 1.b) Import the query set

The query set contains tweets with implicit references to academic papers from the collection set.

The preprocessed query set is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.

In [36]:
# 1) Download the query tweets from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b?ref_type=heads
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_QUERY_TRAIN_DATA = '../subtask4b_query_tweets_train.tsv' #MODIFY PATH
PATH_QUERY_DEV_DATA = '../subtask4b_query_tweets_dev.tsv' #MODIFY PATH
PATH_QUERY_TEST_DATA = '../subtask4b_query_tweets_test.tsv'

In [37]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')
df_query_test = pd.read_csv(PATH_QUERY_TEST_DATA, sep = '\t')

In [38]:
df_query_test.head()

,post_id,tweet_text
0,1,A recent research study published yesterday cl...
1,2,"""We should track the long-term effects of thes..."
2,3,"the agony of ""long haul"" covid-19 symptoms."
3,4,Home and online monitoring and assessment of b...
4,5,"it may be a long one, folks! to avoid exceedin..."


In [ ]:
df_query_train.head()

In [ ]:
df_query_train.info()

In [25]:
df_query_dev.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [26]:
df_query_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     1400 non-null   int64 
 1   tweet_text  1400 non-null   object
 2   cord_uid    1400 non-null   object
dtypes: int64(1), object(2)
memory usage: 32.9+ KB


# 3) Evaluating

In [27]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance


# 4) Embedding

In [28]:
openai.api_key = "XXXX"
EMBED_MODEL   = "text-embedding-3-large"
TOP_K_FINAL   = 10
CORPUS_BATCH  = 500

In [29]:
corpus_texts = (
    df_collection["title"].fillna("").str.strip()
    + "\n\n" +
    df_collection["abstract"].fillna("").str.strip()
).tolist()
uids = df_collection["cord_uid"].tolist()

In [30]:
corpus_embs = []
for i in tqdm(range(0, len(corpus_texts), CORPUS_BATCH), desc="Embedding corpus"):
    #time.sleep(5)
    batch = corpus_texts[i : i + CORPUS_BATCH]
    resp  = openai.embeddings.create(model=EMBED_MODEL, input=batch)
    batch_embs = [d.embedding for d in resp.data]
    corpus_embs.extend(batch_embs)

corpus_embs = normalize(np.array(corpus_embs, dtype=np.float32), axis=1)

Embedding corpus: 100%|██████████| 16/16 [02:23<00:00,  8.99s/it]


In [31]:
claims   = df_query_dev["tweet_text"].tolist()
resp_q   = openai.embeddings.create(model=EMBED_MODEL, input=claims)
query_embs = normalize(
    np.array([d.embedding for d in resp_q.data], dtype=np.float32),
    axis=1
)

In [32]:
results = []
for qi, q_emb in enumerate(tqdm(query_embs, desc="Retrieving")):
    sims     = corpus_embs @ q_emb
    top_idxs = np.argsort(sims)[::-1][:TOP_K_FINAL]
    results.append([uids[i] for i in top_idxs])

Retrieving: 100%|██████████| 1400/1400 [00:03<00:00, 377.95it/s]


In [33]:
df_query_dev["dense_embed_top10"] = results

In [34]:
df_query_dev[["tweet_text","dense_embed_top10"]].head()

,tweet_text,dense_embed_top10
0,covid recovery: this study from the usa reveal...,"[hg3xpej0, jrqlhjsm, rthsl7a9, nksd3wuw, 5hxsa..."
1,"""Among 139 clients exposed to two symptomatic ...","[r58aohnu, tjhtk2up, 1m1hi735, yrowv62k, icgsb..."
2,I recall early on reading that researchers who...,"[sts48u9i, u5nxm9tu, a7frertc, qkg8fwbp, 6l7h3..."
3,You know you're credible when NIH website has ...,"[3sr2exq9, 8j3bb6zx, k0f4cwig, u43jmpyx, xlczc..."
4,Resistance to antifungal medications is a grow...,"[ybwwmyqy, ouvq2wpq, vabb2f26, wdw0a2kl, qh6ri..."


In [35]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'dense_embed_top10')
print(f"Results on the reranked set: {results_reranked}")

Results on the reranked set: {1: np.float64(0.6621428571428571), 5: np.float64(0.7201190476190475), 10: np.float64(0.7257925170068026)}


# 5) Test

In [39]:
claims   = df_query_test["tweet_text"].tolist()
resp_q   = openai.embeddings.create(model=EMBED_MODEL, input=claims)
query_embs = normalize(
    np.array([d.embedding for d in resp_q.data], dtype=np.float32),
    axis=1
)

In [40]:
results = []
for qi, q_emb in enumerate(tqdm(query_embs, desc="Retrieving")):
    sims     = corpus_embs @ q_emb
    top_idxs = np.argsort(sims)[::-1][:5]
    results.append([uids[i] for i in top_idxs])

Retrieving: 100%|██████████| 1446/1446 [00:03<00:00, 393.74it/s]


In [44]:
df_query_test["preds"] = results

In [45]:
df_query_test.head(2)

,post_id,tweet_text,pred,preds
0,1,A recent research study published yesterday cl...,"[bv7hvc1e, pxo11hwp, qgwu9fsk, x4zuv4jo, 26kx1...","[bv7hvc1e, pxo11hwp, qgwu9fsk, x4zuv4jo, 26kx1..."
1,2,"""We should track the long-term effects of thes...","[855atuue, wigakw9h, evf9nz05, gjsrfmj4, in69f...","[855atuue, wigakw9h, evf9nz05, gjsrfmj4, in69f..."


In [47]:
df_query_test[['post_id', 'preds']].to_csv('predictions_embedding_gpt.tsv', index=None, sep='\t')